# Анализ параметров нейронной сети для Bag-of-Words

В данном материале разбираются слои и параметры нейросети на основе Bag-of-Words (BoW), а также рассматриваются методы их оптимизации.

## Введение
Модель BoW использует простые, но эффективные слои для обработки текстовых данных. Корректное понимание функций каждого компонента и методов подбора гиперпараметров критически важно для достижения высокой точности классификации.


## 1. Создание последовательной модели
```python
model_text_bow_softmax = Sequential()
```
- **Что это?**
  - `Sequential()` — контейнер для линейной архитектуры нейросети.
- **Зачем это нужно?**
  - Для простых моделей с последовательной передачей данных. Для сложных архитектур используется функциональный API.


## 2. Первый полносвязный слой
```python
model.add(Dense(200, input_dim=MAX_WORDS_COUNT, activation='relu'))
```
- **Что это?**
  - `Dense(200)` — слой с 200 нейронами.
  - `input_dim` — размерность входных данных (размер словаря BoW).
  - `activation='relu'` — функция активации ReLU.
- **Зачем это нужно?**
  - Обучает веса для входных признаков и формирует скрытое представление данных.


## 3. Слой Dropout
```python
model.add(Dropout(0.25))
```
- **Что это?**
  - Случайное отключение 25% нейронов на каждом шаге обучения.
- **Зачем это нужно?**
  - Предотвращает переобучение за счет уменьшения зависимости модели от отдельных нейронов.


## 4. Слой BatchNormalization
```python
model.add(BatchNormalization())
```
- **Что это?**
  - Нормализация активаций по мини-батчу.
- **Зачем это нужно?**
  - Ускоряет обучение, стабилизирует градиенты и снижает требуемую точность инициализации весов.


## 5. Выходной слой
```python
model.add(Dense(ClassCount, activation='softmax'))
```
- **Что это?**
  - `Dense(ClassCount)` — слой с количеством нейронов = числу классов.
  - `activation='softmax'` — преобразует выходы в вероятности.
- **Зачем это нужно?**
  - Предоставляет вероятностную оценку классов для решения задачи классификации.


## Дополнительные параметры
1. **Оптимизаторы:**
   - `adam` (адаптивный оптимизатор).
   - `sgd` (стохастический градиентный спуск).
2. **Функции потерь:**
   - `categorical_crossentropy` (для многоклассовой классификации).
   - `binary_crossentropy` (для бинарной).
3. **Метрики:**
   - `accuracy` (точность).
4. **Регуляризация:**
   - `kernel_regularizer=l2(0.01)` — L2-регуляризация.
5. **Дополнительные слои:**
   - `GaussianNoise` — добавляет шум к входным данным.
   - `Activation()` — явное указание функции активации.


## Методы подбора параметров
1. **Эмпирический выбор:**
   - Начальные значения (например, `200 нейронов`, `dropout=0.25`).
2. **Grid Search:**
   - Перебор всех возможных комбинаций параметров.
3. **Random Search:**
   - Случайный выбор значений для ускорения оптимизации.
4. **Автоматические инструменты:**
   - **Keras Tuner**, **Optuna**, **Hyperopt**.
5. **Кросс-валидация:**
   - Оценка модели на нескольких подвыборках данных.


## Пример автоматической оптимизации
```python
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from kerastuner.tuners import RandomSearch

def build_model(hp):
    model = Sequential()
    model.add(Dense(
        units=hp.Int('units', min_value=100, max_value=300, step=50),
        input_dim=MAX_WORDS_COUNT,
        activation='relu'))
    model.add(Dropout(
        rate=hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(BatchNormalization())
    model.add(Dense(ClassCount, activation='softmax'))
    model.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='text_classification'
)
tuner.search(X_train, y_train,
            epochs=10,
            validation_data=(X_val, y_val))
best_model = tuner.get_best_models(num_models=1)[0]
```


**## Пример оптимизации для BoW-модели**
### Цель:
Подобрать параметры:
1. `units` (количество нейронов в первом слое).
2. `dropout` (процент отключения нейронов).

### Используемый инструмент:
- **Keras Tuner** — библиотека для автоматической оптимизации.
- **RandomSearch** — алгоритм случайного перебора параметров.


### Этап 1: Импорт библиотек
```python
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from kerastuner.tuners import RandomSearch
```
- **Sequential**: Для создания последовательной модели.
- **Dense/Dropout/BatchNormalization**: Слои нейронной сети.
- **RandomSearch**: Класс для автоматического поиска параметров.


### Этап 2: Функция построения модели
```python
def build_model(hp):
    model = Sequential()
    model.add(Dense(
        units=hp.Int('units', min_value=100, max_value=300, step=50),
        input_dim=MAX_WORDS_COUNT,
        activation='relu'))
    model.add(Dropout(
        rate=hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(BatchNormalization())
    model.add(Dense(ClassCount, activation='softmax'))
    model.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    return model
```
#### Ключевые моменты:
1. **`hp.Int('units', ...)`**:
   - Поиск значения от 100 до 300 с шагом 50.
   - Примеры: 100, 150, 200, 250, 300.
2. **`hp.Float('dropout', ...)`**:
   - Поиск значения от 0.1 до 0.5 с шагом 0.1.
   - Примеры: 0.1, 0.2, ..., 0.5.


### Этап 3: Настройка тюнера
```python
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='text_classification'
)
```
#### Параметры:
1. **`objective='val_accuracy'`**:
   - Модель оптимизируется по точности на валидационных данных.
2. **`max_trials=10`**:
   - Количество уникальных комбинаций параметров для тестирования.
3. **`executions_per_trial=2`**:
   - Каждая комбинация обучается 2 раза для стабилизации результатов.


### Этап 4: Запуск оптимизации
```python
tuner.search(
    X_train, y_train,
    epochs=10,
    validation_data=(X_val, y_val)
)
```
- **`search()`**: Запускает перебор параметров.
- **`epochs=10`**: Каждая модель обучается 10 эпох.


### Этап 5: Получение лучшей модели
```python
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters()[0]
```
- **`get_best_models()`**: Возвращает модель с наилучшей `val_accuracy`.
- **`get_best_hyperparameters()`**: Возвращает оптимальные значения параметров.


## Почему именно эти параметры?
1. **`units` (количество нейронов)**:
   - **100–300**: Баланс между сложностью модели и риском переобучения.
   - Меньше нейронов → проще модель, но возможен недообучение.
   - Больше нейронов → более точная модель, но требует больше ресурсов.
2. **`dropout` (процент отключения)**:
   - **0.1–0.5**: Типичный диапазон для текстовых задач.
   - Высокие значения (0.5) сильнее регуляризуют, но могут уменьшить качество.


## Улучшения и расширения
1. **Добавить другие параметры**:
   - `learning_rate` для оптимизатора `adam`:
     ```python
     optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', [0.001, 0.01]))
     ```
   - `batch_size`:
     ```python
     batch_size=hp.Int('batch_size', min_value=32, max_value=128, step=32)
     ```
2. **Использовать другие алгоритмы**:
   - **BayesianOptimization**: Использует статистические модели для более умного выбора параметров.
   - **Hyperband**: Раннее отсечение слабых моделей для экономии времени.


## Важные замечания
1. **Время выполнения**:
   - Чем больше `max_trials` и `epochs`, тем дольше оптимизация.
   - Для больших моделей используйте `max_trials=5` и `epochs=5` для быстрого тестирования.
2. **Ресурсы**:
   - Требуется GPU/TPU для больших сетей.
3. **Репродуктивность**:
   - Используйте `random_state` для воспроизводимости результатов.
4. **Интерпретация результатов**:
   - Лучшие параметры сохраняются в `best_hyperparameters.values`.


## Пример вывода
После выполнения `tuner.search()` вы увидите:
```
Search: Running search...
Epoch 1/10: loss: 0.6, val_accuracy: 0.82
...
Best val_accuracy: 0.85 (параметры: units=200, dropout=0.3)
```
- **Итог**: Модель с 200 нейронами и dropout=0.3 показала наилучшую точность.


## Итоги
- **Слои:**
  - `Dense` — основа для обучения.
  - `Dropout`, `BatchNormalization` — регуляризация и стабилизация.
  - `softmax` — решение задачи классификации.
- **Подбор параметров:**
  - Используйте комбинацию эмпирического выбора и автоматических инструментов (например, Keras Tuner).
- **Ключевые принципы:**
  - Валидация на тестовых данных.
  - Баланс между сложностью модели и риском переобучения.


## Контрольные вопросы
1. Что такое гиперпараметры и зачем их оптимизируют?
2. Какие параметры подбираются в примере? Для чего?
3. В чем разница между `GridSearch` и `RandomSearch`?
4. Какие параметры можно добавить в функцию `build_model` для улучшения модели?
5. Почему важно использовать `executions_per_trial=2`?


## Домашнее задание
1. Реализуйте оптимизацию параметра `learning_rate` в примере.
2. Сравните результаты `RandomSearch` и `Hyperband` для той же задачи.
3. Объясните, почему dropout=0.5 может быть хуже dropout=0.2 для вашей модели.
---


**# Анализ параметров нейросети с использованием Embedding**



## Анализ модели
Пример модели:
```python
model_text_emb_20 = Sequential()
model_text_emb_20.add(Embedding(input_dim=MAX_WORDS_COUNT, output_dim=20, input_length=WIN_SIZE))
model_text_emb_20.add(SpatialDropout1D(0.2))
model_text_emb_20.add(Flatten())
model_text_emb_20.add(BatchNormalization())
model_text_emb_20.add(Dense(200, activation='relu'))
model_text_emb_20.add(Dropout(0.2))
model_text_emb_20.add(BatchNormalization())
model_text_emb_20.add(Dense(ClassCount, activation='softmax'))
```

### Пошаговое описание слоев:
1. **Embedding**:
   - **`input_dim=MAX_WORDS_COUNT`**: Количество уникальных слов в словаре.
   - **`output_dim=20`**: Размер векторного представления (эмбеддинга) для каждого слова.
   - **`input_length=WIN_SIZE`**: Длина входного текста (например, максимальное количество слов в документе).
   - **Назначение**: Преобразует текст в плотные векторы фиксированной длины.

2. **SpatialDropout1D(0.2)**:
   - **`rate=0.2`**: Отключает 20% единиц в каждом временном шаге (например, слове).
   - **Назначение**: Регуляризация для предотвращения переобучения в последовательностях.

3. **Flatten()**:
   - **Назначение**: Преобразует 3D-выход Embedding в 2D-тензор для подачи в Dense-слои.

4. **BatchNormalization()**:
   - **Назначение**: Нормализует выходы предыдущего слоя, ускоряя обучение и стабилизируя градиенты.

5. **Dense(200, activation='relu')**:
   - **`units=200`**: Количество нейронов в скрытом слое.
   - **`activation='relu'`**: Функция активации для нелинейности.
   - **Назначение**: Обучает сложные комбинации признаков.

6. **Dropout(0.2)**:
   - **`rate=0.2`**: Отключает 20% нейронов на каждом шаге обучения.
   - **Назначение**: Дополнительная регуляризация.

7. **BatchNormalization()**:
   - **Назначение**: Нормализация после Dropout для стабилизации обучения.

8. **Dense(ClassCount, activation='softmax')**:
   - **`units=ClassCount`**: Количество классов.
   - **`activation='softmax'`**: Преобразует выходы в вероятности.
   - **Назначение**: Финальное предсказание класса.


## Дополнительные параметры
1. **Оптимизаторы**:
   - `optimizer='adam'` — адаптивный оптимизатор.
   - `optimizer='sgd'` — стохастический градиентный спуск.
2. **Функции потерь**:
   - `loss='categorical_crossentropy'` — для многоклассовой классификации.
   - `loss='sparse_categorical_crossentropy'` — если метки не закодированы в one-hot.
3. **Метрики**:
   - `metrics=['accuracy']` — оценка точности модели.
4. **Регуляризация**:
   - `kernel_regularizer=l2(0.01)` — L2-регуляризация для весов.
5. **Дополнительные слои**:
   - `LSTM`/`GRU` — для обработки последовательностей.
   - `Conv1D` — для извлечения локальных паттернов.
6. **Параметры Embedding**:
   - `mask_zero=True` — игнорирование пустых значений.
   - `embeddings_initializer='uniform'` — метод инициализации весов.


## Методы подбора гиперпараметров
### 1. Эмпирический выбор
- **Пример**: Начальные значения:
  - `output_dim=20` (размер эмбеддинга).
  - `units=200` (количество нейронов).
  - `dropout=0.2` (примерно 20% регуляризации).

### 2. Grid Search
- **Пример**: Перебор `output_dim` в диапазоне [10, 30] и `dropout` в [0.1, 0.3].

### 3. Random Search
- **Пример**: Случайный выбор значений:
  ```python
  hp.Int('output_dim', min_value=10, max_value=50, step=5)
  hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)
  ```

### 4. Автоматизация с Keras Tuner
- **Пример**: Оптимизация `output_dim`, `units`, `dropout`:
  ```python
  from kerastuner import RandomSearch

  def build_model(hp):
      model = Sequential()
      model.add(Embedding(
          input_dim=MAX_WORDS_COUNT,
          output_dim=hp.Int('output_dim', 10, 50, 5),
          input_length=WIN_SIZE))
      model.add(SpatialDropout1D(hp.Float('spatial_dropout', 0.1, 0.5, 0.1)))
      model.add(Flatten())
      model.add(Dense(
          units=hp.Int('dense_units', 100, 300, 50),
          activation='relu'))
      model.add(Dropout(hp.Float('dropout', 0.1, 0.5, 0.1)))
      model.add(Dense(ClassCount, activation='softmax'))
      model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])
      return model

  tuner = RandomSearch(
      build_model,
      objective='val_accuracy',
      max_trials=10,
      executions_per_trial=2,
      directory='tuning',
      project_name='text_classification'
  )
  tuner.search(X_train, y_train,
              epochs=5,
              validation_data=(X_val, y_val))
  ```
  - **Результат**: Оптимизация по метрике `val_accuracy`.


## Подробный разбор примера автоматизированной оптимизации с Keras Tuner

Приведенный ниже пример демонстрирует, как автоматически подобрать оптимальные значения гиперпараметров для модели с **Embedding-слоем** с помощью **Keras Tuner**. Разберем каждый этап подробно.

### Пример кода:
```python
from kerastuner import RandomSearch

def build_model(hp):
    model = Sequential()
    # Слой Embedding с оптимизируемым output_dim
    model.add(Embedding(
        input_dim=MAX_WORDS_COUNT,
        output_dim=hp.Int('output_dim', min_value=10, max_value=50, step=5),
        input_length=WIN_SIZE
    ))
    # SpatialDropout1D с оптимизируемым процентом отключения
    model.add(SpatialDropout1D(
        hp.Float('spatial_dropout', min_value=0.1, max_value=0.5, step=0.1)
    ))
    model.add(Flatten())
    # Полносвязный слой с оптимизируемым количеством нейронов
    model.add(Dense(
        units=hp.Int('dense_units', min_value=100, max_value=300, step=50),
        activation='relu'
    ))
    model.add(Dropout(
        hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)
    ))
    model.add(Dense(ClassCount, activation='softmax'))
    model.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    return model

# Настройка тюнера
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='tuning',
    project_name='text_classification'
)

# Запуск оптимизации
tuner.search(
    X_train, y_train,
    epochs=5,
    validation_data=(X_val, y_val)
)
```

### Пошаговое объяснение:

#### 1. Функция `build_model(hp)`
- **`hp.Int('output_dim', min_value=10, max_value=50, step=5)`**:
  - Ищет оптимальный размер векторного представления (эмбеддинга) для слов.
  - Возможные значения: 10, 15, 20, ..., 50.
  - **Почему?**:
    - Маленькие значения (10–20) подходят для простых задач с небольшим объемом данных.
    - Большие значения (50) могут улучшить качество, но требуют больше ресурсов.

- **`hp.Float('spatial_dropout', min_value=0.1, max_value=0.5, step=0.1)`**:
  - Оптимизирует процент отключения временных шагов (например, слов) в `SpatialDropout1D`.
  - Возможные значения: 0.1, 0.2, ..., 0.5.
  - **Почему?**:
    - Низкие значения (0.1–0.3) сохраняют больше информации, но рискуют переобучиться.
    - Высокие значения (0.5) сильнее регуляризуют, но могут уменьшить качество.

- **`hp.Int('dense_units', min_value=100, max_value=300, step=50)`**:
  - Ищет оптимальное количество нейронов в первом Dense-слое.
  - Возможные значения: 100, 150, 200, 250, 300.
  - **Почему?**:
    - Больше нейронов → более сложная модель, но требует больше данных.
    - Диапазон 100–300 — баланс между точностью и вычислительными ресурсами.

- **`hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)`**:
  - Оптимизирует процент отключения нейронов в Dense-слое.
  - **Почему?**:
    - Умеренные значения (0.2–0.3) обычно работают лучше для текстовых задач.

#### 2. Настройка тюнера (`RandomSearch`)
- **`objective='val_accuracy'`**:
  - Цель: максимизировать точность на валидационных данных.
- **`max_trials=10`**:
  - Выполняется 10 экспериментов, в каждом из которых случайным образом выбираются параметры.
- **`executions_per_trial=2`**:
  - Каждая комбинация параметров обучается 2 раза для усреднения результатов.
- **`directory='tuning'` и `project_name='text_classification'`**:
  - Папка для сохранения результатов оптимизации.

#### 3. Запуск оптимизации (`tuner.search()`)
- **`epochs=5`**:
  - Каждая модель обучается 5 эпох. Для экономии времени используется небольшое количество эпох на этапе поиска.
- **`validation_data=(X_val, y_val)`**:
  - Проверка качества модели на валидационных данных после каждой эпохи.

#### 4. Результаты
- После завершения, `tuner` выводит таблицу с лучшими комбинациями параметров и их метриками:
  ```
  Search: Running search...
  Epoch 1/5: loss: 0.7, val_accuracy: 0.81
  ...
  Best val_accuracy: 0.85 (параметры: output_dim=30, spatial_dropout=0.2, dense_units=200, dropout=0.25)
  ```
- **Итог**: Лучшая модель использует:
  - Размер эмбеддинга **30**.
  - **20%** отключения в `SpatialDropout1D`.
  - **200 нейронов** в Dense-слое.
  - **25%** Dropout после Dense-слоя.

### Почему именно эти параметры?
1. **`output_dim=30`**:
   - Средний размер эмбеддинга, который балансирует между информативностью и вычислительными затратами.
2. **`spatial_dropout=0.2`**:
   - Низкая регуляризация для текстовых данных, где важны все слова.
3. **`dense_units=200`**:
   - Среднее количество нейронов, позволяющее избежать переобучения и сохранить точность.
4. **`dropout=0.25`**:
   - Умеренная регуляризация для предотвращения переобучения в Dense-слое.

### Улучшения и советы
1. **Добавьте другие параметры**:
   - `learning_rate` для оптимизатора:
     ```python
     optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('lr', [0.001, 0.0001]))
     ```
2. **Используйте другие алгоритмы**:
   - **BayesianOptimization** для более умного выбора параметров.
   - **Hyperband** для ранней остановки низкоэффективных моделей.
3. **Проверьте стабильность**:
   - Увеличьте `executions_per_trial=3` для более надежных результатов.

### Ошибки и их исправление
1. **Медленный поиск**:
   - Уменьшите `max_trials` или `epochs` для экономии времени.
2. **Переобучение**:
   - Увеличьте `spatial_dropout` или `dropout`.
3. **Низкая точность**:
   - Увеличьте `output_dim` или `dense_units`.
   - Добавьте дополнительные слои (например, `Conv1D`).

### Важные замечания
- **Время выполнения**: Для 10 экспериментов с 5 эпохами на каждом этапе может потребоваться несколько минут.
- **Репродуктивность**: Используйте `random_state` для воспроизводимости результатов.
- **Глубина модели**: Для сложных задач можно добавить дополнительные Dense-слои с оптимизацией их параметров.


## Пример разбора модели
### 1. **Embedding**:
- **`output_dim=20`**:
  - Маленькие значения (10–50) часто используются для текстов небольшого объема.
  - Большие значения (100–300) требуют больше данных, но могут улучшить качество.

### 2. **SpatialDropout1D(0.2)**:
- **Почему 0.2?**:
  - Низкий уровень регуляризации для текстовых данных, где потеря части слов может быть полезной.
  - Высокие значения (0.5+) могут привести к потере важной информации.

### 3. **Flatten()**:
- **Назначение**: Сжатие 3D-входа (batch_size, sequence_length, embedding_dim) в 2D (batch_size, features).
- **Альтернатива**: Использование `GlobalAveragePooling1D` или `GlobalMaxPooling1D` для усреднения/максимизации по временной оси.

### 4. **BatchNormalization()**:
- **Почему дважды?**:
  - После Flatten() — для стабилизации после SpatialDropout.
  - После Dense() — для нормализации перед Dropout.

### 5. **Dense(200, activation='relu')**:
- **`units=200`**:
  - Баланс между сложностью и вычислительными ресурсами.
  - Можно попробовать 100–300 для разных задач.

### 6. **Dropout(0.2)**:
- **Почему 0.2?**:
  - Умеренная регуляризация для предотвращения переобучения в Dense-слое.
  - Для больших сетей можно увеличить до 0.3–0.5.


## Контрольные вопросы
1. Зачем используется слой `Embedding` в модели?
2. В чем разница между `Dropout` и `SpatialDropout1D`?
3. Какие параметры можно оптимизировать в слое `Embedding`?
4. Почему применяется `Flatten()` после `Embedding`?
5. Какие алгоритмы можно использовать для автоматизированной оптимизации гиперпараметров?


## Домашнее задание
1. Измените параметр `output_dim` в Embedding и сравните точность модели.
2. Добавьте в модель слой `Conv1D` и настройте его параметры.
3. Реализуйте оптимизацию `output_dim` и `units` с помощью Keras Tuner.
4. Объясните, почему `BatchNormalization` полезен перед активацией, а не после.
---


## **Продвинутые методы классификации текста**
Список методов, превосходящих простые BoW и базовые embedding:

| Метод                          | Описание                                                                |
|-------------------------------|--------------------------------------------------------------------------|
| **RNN (Recurrent Neural Networks)**       | Обрабатывают последовательности, сохраняя контекст           |
| **LSTM/GRU**                   | Разновидности RNN, решающие проблему долгой зависимости                 |
| **CNN (Convolutional Neural Networks)**   | Ищут локальные признаки в тексте через фильтры               |
| **Transformers**               | Работают с самозависимыми attention-механизмами (например, BERT, GPT)   |
| **BRNN (Bidirectional RNN)**   | Анализируют контекст в обоих направлениях                               |
| **N-gram-based networks**       | Используют N-граммы без явного токенизации                             |
| **Hybrid models**              | Комбинации embedding + CNN/RNN/Transformers для улучшения точности      |

Каждый метод решает задачи контекста, семантики или масштабируемости.

---


# Метод Transformers: подробное объяснение
## Основы
1. **Проблема предыдущих моделей**:
   - RNN/GRU/LSTM сталкивались с проблемой долгой зависимости (long-term dependency) и медленной обработкой из-за последовательного характера [1](https://medium.com/towards-data-science/transformers-141e32e69591).
   - CNN могли параллельно обрабатывать данные, но плохо учитывали глобальный контекст [6](https://medium.com/@punya8147_26846/differences-between-transformers-and-traditional-sequence-models-e53c7e1b849a).

2. **Ключевая идея Transformers**:
   - **Self-attention**: Механизм, который позволяет модели учитывать влияние всех слов в предложении на каждое конкретное слово [1](https://medium.com/towards-data-science/transformers-141e32e69591).
   - **Параллельная обработка**: Вместо обработки последовательности пошагово, обрабатываются все токены одновременно [6](https://medium.com/@punya8147_26846/differences-between-transformers-and-traditional-sequence-models-e53c7e1b849a).

3. **Основные компоненты**:
   - **Encoder**: Преобразует входные токены в векторы, учитывая контекст [3](https://www.datacamp.com/tutorial/how-transformers-work),[4](https://medium.com/@nimritakoul01/the-transformer-model-all-components-simply-explained-series-38c76ecdeeef).
   - **Decoder**: Используется для генерации выходных последовательностей (например, в машинном переводе) [4](https://medium.com/@nimritakoul01/the-transformer-model-all-components-simply-explained-series-38c76ecdeeef), [9](https://myscale.com/blog/key-components-transformer-models-understanding/).
   - **Self-attention layer**: Вычисляет важность каждого токена относительно других [2](https://www.quora.com/What-are-the-key-components-of-the-Transformer-architecture-and-how-do-they-work-in-ChatGPT), [10](https://lih-verma.medium.com/components-of-transformer-architecture-748f74a1a40d).
   - **Positional encoding**: Добавляет информацию о позиции токена, так как сам attention не учитывает порядок [9](https://myscale.com/blog/key-components-transformer-models-understanding/), [10](https://lih-verma.medium.com/components-of-transformer-architecture-748f74a1a40d).

4. **Как работает self-attention**:
   - Для каждого токена вычисляются три вектора: Query (запрос), Key (ключ), Value (значение).
   - Важность токена определяется через скалярное произведение Query и Key других токенов.
   - Итоговый вектор формируется как взвешенная сумма Value-векторов [2](https://www.quora.com/What-are-the-key-components-of-the-Transformer-architecture-and-how-do-they-work-in-ChatGPT), [10](https://lih-verma.medium.com/components-of-transformer-architecture-748f74a1a40d).


## Общий синтаксис и архитектура
### Структура модели:
1. **Encoder**:
   - Слой **self-attention**: Вычисляет важность токенов.
   - Слой **feed-forward**: Обрабатывает векторы через полносвязные слои для извлечения признаков [2](https://www.quora.com/What-are-the-key-components-of-the-Transformer-architecture-and-how-do-they-work-in-ChatGPT), [5](https://massedcompute.com/faq-answers/?question=What+are+the+key+differences+between+the+architecture+of+transformer-based+and+non-transformer+based+large+language+models%3F).
   - **Norm и residual connection**: Стабилизируют обучение [10]().

2. **Decoder** (используется в задачах генерации):
   - Слой **masked self-attention**: Учитывает только предыдущие токены при генерации [4](https://medium.com/@nimritakoul01/the-transformer-model-all-components-simply-explained-series-38c76ecdeeef).
   - Слой **encoder-decoder attention**: Учитывает информацию из encoder-а [9](https://myscale.com/blog/key-components-transformer-models-understanding/).

3. **Multi-head attention**:
   - Параллельно вычисляет несколько attention-векторов для лучшего захвата контекста [2](https://www.quora.com/What-are-the-key-components-of-the-Transformer-architecture-and-how-do-they-work-in-ChatGPT), [7](https://www.portotheme.com/what-are-the-key-differences-between-transformer-models-and-traditional-neural-networks/).

### Пример архитектуры BERT:
   - Только encoder (поскольку задача классификации, а не генерации).
   - 12 или 24 слоев encoder.
   - Использует маскирование для обучения на замаскированных токенах [3](https://www.datacamp.com/tutorial/how-transformers-work).


## Задача классификации тональности текста
**Цель**: Определить, является ли отзыв на фильм положительным или отрицательным.
**Датасет**: IMDb movie reviews (25,000 примеров).

### Этапы решения:
1. **Подготовка данных**: Разделение на train/val/test.
2. **Токенизация**: Преобразование текста в тензоры через BERT-токенизатор.
3. **Обучение**: Использование предобученной модели BERT.
4. **Оценка**: Вычисление точности и F1-меры.


## Код для решения задачи
```python
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import load_dataset
from torch.utils.data import DataLoader

# Загрузка датасета
dataset = load_dataset('imdb')
train_dataset = dataset['train']
test_dataset = dataset['test']

# Инициализация токенизатора и модели
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2  # 2 класса: positive/negative
)

# Функция токенизации
def tokenize(batch):
    return tokenizer(batch['text'],
                    padding=True,
                    truncation=True,
                    max_length=512)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

# Конвертация в DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

# Обучение
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

for epoch in range(3):
    model.train()
    for batch in train_loader:
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Оценка на тесте
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for batch in test_loader:
            inputs = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**inputs)
            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == batch['label'].to(device)).sum().item()
            total += len(batch['label'])
        print(f'Epoch {epoch}: Accuracy = {correct/total:.4f}')
```

### Пояснение кода:
1. **Токенизация**:
   - `tokenizer` преобразует текст в числовые токены и добавляет специальные токены `[CLS]` и `[SEP]` [3](https://www.datacamp.com/tutorial/how-transformers-work).
2. **Модель BERT**:
   - `BertForSequenceClassification` добавляет классификатор на основе выхода encoder-а [3](https://www.datacamp.com/tutorial/how-transformers-work).
3. **Обучение**:
   - `loss.backward()` вычисляет градиенты через attention-механизм и полносвязные слои [10](https://lih-verma.medium.com/components-of-transformer-architecture-748f74a1a40d).
4. **Оценка**:
   - `torch.argmax` выбирает класс с максимальной вероятностью из выхода softmax [3](https://www.datacamp.com/tutorial/how-transformers-work).


## Пример работы кода
Пусть у нас есть отзыв:
```
text = 'This movie was incredibly boring and predictable.'
```
1. **Токенизация**:
   - Текст преобразуется в: `[101, 1244, 2113, 1107, 1226, 1110, 1190, 102]`.
2. **Обработка модели**:
   - **Self-attention**: Каждый токен оценивается относительно других.
   - **Feed-forward**: Извлекаются признаки, указывающие на негативный тон.
3. **Выход**: Модель вернет `label=1` (отрицательный).

### Ключевые шаги в коде:
1. **masked self-attention**: Не используется, так как задача классификации [4](https://medium.com/@nimritakoul01/the-transformer-model-all-components-simply-explained-series-38c76ecdeeef).
2. **Positional encoding**: Добавляется автоматически в BERT [8](https://t.me/s/rybolos_channel?before=569), [9](https://myscale.com/blog/key-components-transformer-models-understanding/).
3. **Классификация**: Выход encoder-а подается на полносвязный слой для предсказания [3](https://www.datacamp.com/tutorial/how-transformers-work).


## Задача для закрепления материала
**Задача**: Классификация новостей по темам (например, спорт, политика, технологии).

### Требования:
1. Используйте датасет **AG News** (4 класса, 120k примеров).
2. Примените модель **DistilBERT** (упрощенный вариант BERT).
3. Оцените модель через **F1-меру**.

### Подсказки:
1. Загрузите датасет через `load_dataset('ag_news')`.
2. Используйте `DistilBertTokenizer` и `DistilBertForSequenceClassification`.
3. Измените `num_labels=4`.

### Пример кода:
```python
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=4  # 4 класса
)
```

### Ожидаемый результат:
Точность на тесте должна быть >85%.

---


# **Метод Hybrid Models: подробное объяснение**
## Основы
1. **Что такое Hybrid Models**:
   - Комбинация **Embedding** + **CNN/RNN/Transformers** для объединения преимуществ разных подходов [7](https://www.sciencedirect.com/science/article/pii/S277267112400216X), [9]().
   - Например: Embedding для векторизации → CNN для локальных паттернов → LSTM для контекста [6](https://www.sciencedirect.com/science/article/abs/pii/S0378779622001389).

2. **Преимущества**:
   - **Локальные признаки (CNN)**: Извлекают ключевые фразы и нграммы [5](https://machinelearningmastery.com/cnn-long-short-term-memory-networks/).
   - **Контекст (RNN/Transformers)**: Учитывают связь между словами в предложении [13](https://huggingface.co/blog/rwkv).
   - **Глобальные признаки (Embedding)**: Предоставляют базовые представления слов [1](https://www.cloudflare.com/ru-ru/learning/ai/what-are-embeddings/), [2](https://habr.com/ru/companies/otus/articles/787116/).

3. **Примеры архитектур**:
   - **CNN + LSTM**: CNN извлекает фичи, LSTM анализирует их последовательность [5](https://machinelearningmastery.com/cnn-long-short-term-memory-networks/).
   - **Embedding + Transformer**: Базовые векторы + глобальный контекст через attention [9](https://arxiv.org/abs/2312.11825), [14](https://medium.com/@data-overload/unlocking-sequential-understanding-recurrent-neural-networks-rnns-and-transformers-dcf83fbe0c9).
   - **Hybrid BERT + RNN**: Усилить глобальный контекст предобученной модели [4](https://medium.com/@pandeyarpit88/building-a-hybrid-architecture-harnessing-the-power-of-microservices-and-monoliths-80f79b310d75).


## Общий синтаксис и архитектура
### Пример: Hybrid model для классификации (Embedding + CNN + LSTM):
1. **Embedding layer**:
   - Преобразует слова в векторы фиксированной длины [2](https://habr.com/ru/companies/otus/articles/787116/).
2. **CNN layers**:
   - **Conv1D**: Извлекает локальные признаки (например, эмоциональные выражения) [8](https://medium.com/@mijanr/different-ways-to-combine-cnn-and-lstm-networks-for-time-series-classification-tasks-b03fc37e91b6).
   - **MaxPooling1D**: Снижает размерность, сохраняя ключевые фичи [8](https://medium.com/@mijanr/different-ways-to-combine-cnn-and-lstm-networks-for-time-series-classification-tasks-b03fc37e91b6).
3. **RNN/LSTM layer**:
   - Обрабатывает последовательность фич из CNN для уловия контекста [3](https://link.springer.com/chapter/10.1007/978-3-031-19608-9_4).
4. **Dense layers**:
   - Полносвязные слои для финальной классификации [2](https://habr.com/ru/companies/otus/articles/787116/).
5. **Регуляризация**:
   - Dropout и BatchNormalization для предотвращения переобучения [3](https://link.springer.com/chapter/10.1007/978-3-031-19608-9_4).

### Пример архитектуры:
```python
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=100, input_length=max_length),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=4),
    LSTM(64),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])
```
### Пояснение:
1. **Embedding**: Векторы слов размером 100 [2](https://habr.com/ru/companies/otus/articles/787116/).
2. **Conv1D**: Фильтры размером 5 для локальных паттернов [10](https://link.springer.com/article/10.1007/s11063-024-11687-w).
3. **LSTM**: Обрабатывает 64 нейрона для контекста [10](https://link.springer.com/article/10.1007/s11063-024-11687-w).
4. **Dropout**: 50% нейронов отключаются для регуляризации [11](https://www.researchgate.net/figure/Hybrid-CNN-LSTM-model_fig5_355373442).


## Задача классификации тональности текста
**Цель**: Определить, является ли отзыв на фильм положительным или отрицательным.
**Датасет**: IMDb movie reviews (25,000 примеров).

### Этапы решения:
1. **Подготовка данных**: Токенизация, разбиение на train/val/test.
2. **Построение Hybrid model**:
   - Embedding → CNN → LSTM → Dense.
3. **Обучение**: Использование категориальной кросс-энтропии.
4. **Оценка**: Точность и матрица ошибок.


## Код для решения задачи
```python
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout

# Загрузка данных
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=5000)
max_length = 500
X_train = pad_sequences(X_train, maxlen=max_length)
X_test = pad_sequences(X_test, maxlen=max_length)

# Построение Hybrid model
model = Sequential([
    Embedding(input_dim=5000, output_dim=100, input_length=max_length),
    Conv1D(128, 5, activation='relu'),
    MaxPooling1D(4),
    LSTM(64),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # 2 класса → sigmoid
])

# Компиляция и обучение
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])
history = model.fit(X_train, y_train,
                   epochs=5,
                   validation_split=0.2)

# Оценка
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Accuracy: {test_acc:.4f}')
```

### Пояснение кода:
1. **Embedding**: 5000 слов → векторы размера 100 [2](https://habr.com/ru/companies/otus/articles/787116/).
2. **Conv1D**: Фильтры 5x128 ищут локальные паттерны [12](https://www.nature.com/articles/s41598-024-73452-2).
3. **MaxPooling**: Снижает размерность до 124 (500-5+1)/4 ≈ 124 [12](https://www.nature.com/articles/s41598-024-73452-2).
4. **LSTM**: Обрабатывает 64 нейрона для глобального контекста [12](https://www.nature.com/articles/s41598-024-73452-2).
5. **Dropout**: 50% отключения нейронов для регуляризации [12](https://www.nature.com/articles/s41598-024-73452-2).
6. **Sigmoid**: Для бинарной классификации [2](https://habr.com/ru/companies/otus/articles/787116/).


## Пример работы кода
Пусть у нас есть отзыв:
```
text = 'This movie was incredibly boring and predictable.'
```
1. **Токенизация**:
   - Преобразуется в последовательность индексов (например, [2, 56, 1000, ...]).
2. **Embedding**:
   - Каждый индекс → вектор длиной 100.
3. **CNN**:
   - Фильтры анализируют окна из 5 слов, выделяя признаки вроде 'boring' и 'predictable'.
4. **LSTM**:
   - Учитывает связь между фичами, например, 'incredibly' усиливает негатив [12](https://www.nature.com/articles/s41598-024-73452-2).
5. **Выход**: Вероятность 0.9 для класса 'negative'.

### Ключевые шаги:
1. **Conv1D**: Извлекает фичи типа 'negative phrases'.
2. **LSTM**: Соединяет локальные фичи в глобальный контекст.
3. **Dropout**: Предотвращает переобучение на шумовых фичах.


## Задача для закрепления материала
**Задача**: Классификация новостей по категориям (спорт, политика, технологии).

### Требования:
1. Используйте датасет **Reuters** (10 классов, ~10k примеров).
2. Постройте Hybrid model с **CNN + GRU**.
3. Добавьте **BatchNormalization** после каждого слоя.

### Подсказки:
1. Загрузите данные через `keras.datasets.reuters`.
2. Архитектура:
   ```python
   model = Sequential([
       Embedding(...),
       Conv1D(...),
       GRU(32),
       Dense(...)
   ])
   ```
3. Оптимизатор: Adam, learning_rate=0.001.

### Ожидаемый результат:
Точность на тесте > 80%.

---